`importing necessary libraries`

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
!pip install -U accelerate
!pip install -U transformers
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 32.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


importing data with little changes

In [ ]:
#use cleaned data from the LSTM model
data = pd.read_csv('/content/drive/MyDrive/ZUM/projekt/data2_clean.csv', usecols=['text', 'target'], index_col=False)

In [ ]:
#column names normalization
data.rename(columns={ 'target':'label'}, inplace=True)

In [ ]:
data

,text,label
0,kicker watchlist xide tit soq pnk cpw bpz aj t...,1
1,user aap movie number return feageed indicator...,1
2,user I have afraid short amzn looking like nea...,1
3,mnta number,1
4,oi number,1
...,...,...
5786,industry body cii say discom likely suffer net...,0
5787,gold price slip rs number investor book profit...,0
5788,worker bajaj auto agree number wage cut period...,1
5789,sharemarket live sensex day high number point ...,1


In [ ]:
data[data['text'].isna()]

,text,label
1250,NaN,1


In [ ]:
data.dropna(inplace=True)

In [ ]:
from datasets import Dataset

dataset_ = Dataset.from_pandas(data)
dataset_ = dataset_.remove_columns(["__index_level_0__"])
dataset = dataset_.train_test_split(0.1)

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5211
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 579
    })
})


In [ ]:
dataset['train'][0]

{'text': 'maruti suzuki march sale plunge nearly number amid coronavirus outbreak httpstco05lzqly97h',
 'label': 0}

Fine-tuning

In [ ]:
#using roBERTa model instead of BERT
model_checkpoint = 'roberta-base'
batch_size = 128

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#function which will make tokens from text in dataset
def process(x):
  return tokenizer(x['text'])

In [ ]:
dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 5211
})

In [ ]:
train_ds = dataset['train'].map(process)
test_ds = dataset['test'].map(process)

Map:   0%|          | 0/5211 [00:00<?, ? examples/s]

Map:   0%|          | 0/579 [00:00<?, ? examples/s]

In [ ]:
train_ds

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 5211
})

In [ ]:
#dealing with model definition
#i used hugging face libraries specifically transformers
from transformers.pipelines.base import AutoModel
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

#I have to classes to predict
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    f'{model_checkpoint}_sentiment_analysis',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = 5,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy'
)

In [ ]:
from datasets import load_metric
import numpy as np

metric = load_metric('accuracy')

def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate()

{'eval_loss': 0.738693356513977,
 'eval_accuracy': 0.40241796200345425,
 'eval_runtime': 1.2659,
 'eval_samples_per_second': 457.383,
 'eval_steps_per_second': 3.95}

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.604824,0.685665
2,No log,0.565655,0.749568
3,No log,0.542565,0.759931
4,No log,0.530937,0.768566
5,No log,0.543759,0.768566


TrainOutput(global_step=205, training_loss=0.47207139178020197, metrics={'train_runtime': 223.6126, 'train_samples_per_second': 116.518, 'train_steps_per_second': 0.917, 'total_flos': 467756984219400.0, 'train_loss': 0.47207139178020197, 'epoch': 5.0})

In [ ]:
trainer.model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
import torch

# text = 'This is a positive example'
text = 'The stocks will raise in next 2 years' #type text which sentiment will be predicted
inputs = tokenizer(text, return_tensors='pt')
device='cuda' if torch.cuda.is_available() else 'cpu'
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

In [ ]:
#saving model in case using it in different problems
torch.save(model.state_dict(), 'robert_model.pt')

In [ ]:
with torch.no_grad():
  outputs = model(input_ids=input_ids, attention_mask=attention_mask)
  logits = outputs.logits
  predictions = torch.argmax(logits, dim=-1)

if predictions.item() == 0:
  print('This is a negative text')
else:
  print('This is a positive text')

This is a positive text
